# EVAC Assessment 2 - Society-based Cooperation

In [1]:
import random
from enums import Society
from deap import creator, base, tools
import itertools
import logging
import numpy as np

logging.basicConfig(level=logging.INFO) # Initializes the logging level used to output to console

In [2]:
OUTCOMES = {
    # Saints cooperate with everyone
    (Society.SAINTS, Society.SAINTS):           (4,4), # Both cooperate
    (Society.SAINTS, Society.BUDDIES):          (0,6), # Buddies are selfish
    (Society.SAINTS, Society.FIGHT_CLUB):       (4,4), # Both cooperate
    (Society.SAINTS, Society.VANDALS):          (0,6), # Vandals are selfish

    # Buddies only cooperate with each other
    (Society.BUDDIES, Society.SAINTS):          (6,0), # Buddies are selfish
    (Society.BUDDIES, Society.BUDDIES):         (4,4), # Both cooperate
    (Society.BUDDIES, Society.FIGHT_CLUB):      (6,0), # Buddies are selfish
    (Society.BUDDIES, Society.VANDALS):         (1,1), # Both selfish
    
    # Fight club cooperate with everyone but themselves
    (Society.FIGHT_CLUB, Society.SAINTS):       (4,4), # Both cooperate
    (Society.FIGHT_CLUB, Society.BUDDIES):      (0,6), # Buddies are selfish
    (Society.FIGHT_CLUB, Society.FIGHT_CLUB):   (1,1), # Both selfish
    (Society.FIGHT_CLUB, Society.VANDALS):      (0,6), # Vandals are selfish
    
    # Vandals cooperate with no one
    (Society.VANDALS, Society.SAINTS):          (6,0), # Vandals are selfish
    (Society.VANDALS, Society.BUDDIES):         (1,1), # Both selfish
    (Society.VANDALS, Society.FIGHT_CLUB):      (6,0), # Vandals are selfish
    (Society.VANDALS, Society.VANDALS):         (1,1), # Both selfish
}

HISTORY_LOOKUP = {y: x for x, y in dict(enumerate(itertools.product([society.value for society in Society], repeat=6))).items()}

In [3]:
IND_SIZE = 4**6 # will actually be 4^6

def reset_population(population):
    for indiv in population:
        indiv.total_score = 0
        indiv.rounds_played = 0
        indiv.society = random.choice(list(Society))
        indiv.history = [random.choice([society.value for society in Society]) for i in range(6)]


def play_round(indiv1, indiv2):
    indiv1.rounds_played += 1
    indiv2.rounds_played += 1
    
    payoffs = OUTCOMES[(indiv1.society, indiv2.society)]
    indiv1.total_score += payoffs[0]
    indiv2.total_score += payoffs[1]

    new_match = [indiv1.society.value, indiv2.society.value]
    indiv1.history = indiv1.history[2:6] + new_match
    indiv2.history = indiv2.history[2:6] + new_match[::-1]

    chr1_index = HISTORY_LOOKUP[tuple(indiv1.history)]
    indiv1.society = Society(indiv1[chr1_index])

    chr2_index = HISTORY_LOOKUP[tuple(indiv2.history)]
    indiv2.society = Society(indiv1[chr2_index])

def evaluate_agents(population, num_rounds):
    logging.info("Running the game")
    for i in range(num_rounds):
        indiv1 = random.choice(population)
        indiv2 = random.choice(population)
        while indiv2 == indiv1:
            indiv2 = random.choice(population)
        play_round(indiv1, indiv2)
    
    for ind in population:
        if ind.rounds_played == 0:
            ind.fitness.values = 0,
        else:
            ind.fitness.values = ind.total_score / ind.rounds_played,

def print_indiv(indiv):
    print("~~~~~~~~~~~~~~~~~~~~~~~")
    print(f"{indiv.total_score=}")
    print(f"{indiv.rounds_played=}")
    print(f"{indiv.society=}")
    print(f"{indiv.history=}")
    print("~~~~~~~~~~~~~~~~~~~~~~~")

def run_GA(gen_num=200, pop_num=20, round_num=400, mut_prob=0.021, cx_prob=0.15, tourn_size=5, headless=True):
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax, total_score=0, rounds_played=0, society=Society.SAINTS, history = [0 for i in range(6)])

    toolbox = base.Toolbox()
    toolbox.register("attr_int", random.choice, [society.value for society in Society])
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, n = IND_SIZE)

    toolbox.register("evaluate", evaluate_agents)

    # Registers function to select, mate and mutate individuals
    toolbox.register("select", tools.selTournament, tournsize=tourn_size)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutShuffleIndexes, indpb=mut_prob)

    toolbox.register("mutateShuffleIndexes", tools.mutShuffleIndexes, indpb = 0.05)

    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("reset_population", reset_population)

    # Registers the statistics & logbook that will be logged during the GA
    stats = tools.Statistics(key=lambda ind: ind.fitness.values)
    stats.register("mean", np.mean)
    stats.register("std", np.std)
    stats.register("median", np.median)
    stats.register("min", np.min)
    stats.register("max", np.max)
    logbook = tools.Logbook()

    population = toolbox.population(pop_num)
    toolbox.reset_population(population)

    toolbox.evaluate(population, round_num)

    # Genetic Algorithm
    for g in range(gen_num):
        logging.debug("Running generation " + str(g))

        # Selects number of individuals equal to population length
        offspring = toolbox.select(population, len(population))
        # Includes duplicates so clones all individuals
        offspring = list(map(toolbox.clone, offspring))

        print([child.fitness.values[0]] for child in offspring)

        # Performs crossover on 2 individuals based on previously defined probability
        for indiv1, indiv2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < cx_prob:
                toolbox.mate(indiv1, indiv2)

        # Mutates offspring based on previously defined probability #TODO: Modify probability/algorithm type?
        for mutant in offspring:
            toolbox.mutate(mutant)
        
        toolbox.reset_population(offspring)

        # Recalculates fitness values for mutated offspring
        toolbox.evaluate(offspring, round_num)

        # Replaces old population with new mutated offspring
        population[:] = offspring

        # Compiles & records the statistics for the new generation
        record = stats.compile(population)
        logging.info(record)
        logbook.record(gen=g, **record)

    return logbook, population


In [3]:
run_GA(gen_num=150, pop_num=2000, round_num=5000, mut_prob=0.021, cx_prob=0.15, tourn_size=50, headless=True)

NameError: name 'run_GA' is not defined